In [1]:
import pymania as mn
import pylab as plt
from collections import Counter
from py2neo import Node, Relationship, Graph
import numpy

graph = Graph(host="canopus.cc.gatech.edu",password='1234')

In [2]:
query = '''MATCH (n:ROI)-[r:MANIA2]->(m:ROI)
WHERE r.SUBJECT IN [126426, 137431, 144125, 146735, 152427, 153227, 177140, 180533, 186545, 188145] AND n.name STARTS WITH 'L' AND r.is_connected=TRUE
WITH n.name AS source, m.name AS target, r.SUBJECT AS subject, r.correction_type as correction_type
WITH source, target, COUNT(*) AS connections, COLLECT(subject) AS subjects, COLLECT(correction_type) as correction_types WHERE connections=5
RETURN source, target, subjects, correction_types order by source, target'''
A1 = graph.run(query).data()
query = '''MATCH (n:ROI)-[r:MANIA2]->(m:ROI)
WHERE r.SUBJECT IN [126426, 137431, 144125, 146735, 152427, 153227, 177140, 180533, 186545, 188145] AND n.name STARTS WITH 'L' AND r.is_connected=False
WITH n.name AS source, m.name AS target, r.SUBJECT AS subject, r.correction_type as correction_type
WITH source, target, COUNT(*) AS connections, COLLECT(subject) AS subjects, COLLECT(correction_type) as correction_types WHERE connections=5
RETURN source, target, subjects, correction_types order by source,target'''
A2 = graph.run(query).data()
l = len(A1)
D = {}
for i in range(l):
    rec1 = A1[i]
    rec2 = A2[i]
    r1 = rec1['source']
    r2 = rec1['target']
    if (r1 != rec2['source'] or r2 != rec2['target']):
        print('wrong data')
    tmp1 = rec1['correction_types'] + rec2['correction_types']
    tmp2 = Counter(tmp1)
    a = tmp2.most_common(1)[0]
    D[(r1,r2)] = a[0]
Counter(D.values())

Counter({'envelope': 797, 'fallback': 55})

In [3]:
# pick 5 fallback, 3 envelope and 2 null
import random
t = list(D.keys())
random.shuffle(t)
f = 0
e = 0
C = []
for w in t:
    if D[w] == 'fallback' and f<2:
        C.append(w)
        f = f + 1
    if D[w] == 'envelope' and e<8:
        C.append(w)
        e = e + 1

In [7]:
import pickle as pk
with open('train10.pk','wb') as f:
    pk.dump(C,f)